In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

x = 2016
y = 2017

jornadas_por_temporada = {
    (1928, 1934): 18,
    (1934, 1936): 22,
    (1936, 1939): 0,
    (1939, 1941): 22,
    (1941, 1950): 26,
    (1951, 1970): 30,
    (1971, 1985): 34,
    (1987, 1995): 38,
    (1996, 2020): 38,
    (2021, 2025): 38,
}

def get_jornadas(x, y):
    if x == 1986 and y == 1987:
        return 0
    for (inicio, fin), jornadas in jornadas_por_temporada.items():
        if inicio <= x < fin:
            return jornadas
    return 0

def getURL_jornadas(x, y, añoFinal):
    while y <= añoFinal:
        jornadaMaxima = get_jornadas(x, y)

        if jornadaMaxima == 0:
            print(f"Temporada {x}/{y} excluida o sin jornadas.")
        else:
            datos_totales = []  # Aquí juntaremos todas las jornadas de la temporada

            for jornada in range(1, jornadaMaxima + 1):
                #url = f"https://www.livefutbol.com/calendario/esp-primera-division-{x}-{y}_2-spieltag/{jornada}/"
                url = f"https://www.livefutbol.com/todos_partidos/esp-primera-division-{x}-{y}_2/"
                headers = {'User-Agent': 'Mozilla/5.0'}
                response = requests.get(url, headers=headers)
                soup = BeautifulSoup(response.content, 'html.parser')

                tabla = soup.find('table', class_='standard_tabelle')

                jornada_actual = f"Jornada {jornada} ({x}/{y})"

                for fila in tabla.find_all('tr'):
                    columnas = fila.find_all('td')
                    if len(columnas) == 7:
                        fila_datos = [col.get_text(strip=True) for col in columnas]
                        fila_datos.insert(0, jornada_actual)
                        datos_totales.append(fila_datos)

            # Guardar una vez por temporada
            columnas = ['Jornada', 'Fecha', 'Hora', 'Local', 'Resultado', 'Visitante', 'Asistencia', 'Estadio']
            df = pd.DataFrame(datos_totales, columns=columnas)

            nombre_archivo = f"Ligas_Resultados_CSV/laliga_{x}_{y}.csv"
            df.to_csv(nombre_archivo, index=False, encoding='utf-8')

            print(f"Temporada {x}/{y} guardada con {len(df)} partidos.")

        x += 1
        y += 1

getURL_jornadas(2016, 2017, 2017)


Temporada 2016/2017 guardada con 14440 partidos.


In [5]:
def getURLS(x, añoFinal):
    y = x + 1
    urls = []
    while y <= añoFinal:
        if 1936 <= x <= 1938 or x == 1986:
            x += 1
            y += 1
            continue
        url = f"https://www.livefutbol.com/todos_partidos/esp-primera-division-{x}-{y}"
        urls.append(url)
        x += 1
        y += 1
    return urls

urls_generadas = getURLS(1929, 2024)
print(urls_generadas)


['https://www.livefutbol.com/todos_partidos/esp-primera-division-1929-1930', 'https://www.livefutbol.com/todos_partidos/esp-primera-division-1930-1931', 'https://www.livefutbol.com/todos_partidos/esp-primera-division-1931-1932', 'https://www.livefutbol.com/todos_partidos/esp-primera-division-1932-1933', 'https://www.livefutbol.com/todos_partidos/esp-primera-division-1933-1934', 'https://www.livefutbol.com/todos_partidos/esp-primera-division-1934-1935', 'https://www.livefutbol.com/todos_partidos/esp-primera-division-1935-1936', 'https://www.livefutbol.com/todos_partidos/esp-primera-division-1939-1940', 'https://www.livefutbol.com/todos_partidos/esp-primera-division-1940-1941', 'https://www.livefutbol.com/todos_partidos/esp-primera-division-1941-1942', 'https://www.livefutbol.com/todos_partidos/esp-primera-division-1942-1943', 'https://www.livefutbol.com/todos_partidos/esp-primera-division-1943-1944', 'https://www.livefutbol.com/todos_partidos/esp-primera-division-1944-1945', 'https://ww

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.livefutbol.com/todos_partidos/esp-primera-division-1995-1996/"
headers = {'User-Agent': 'Mozilla/5.0'}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

tabla = soup.find('table', class_='standard_tabelle')

datos = []
for fila in tabla.find_all('tr'):
    columnas = fila.find_all('td')
    if len(columnas) == 7:
        fila_datos = [col.get_text(strip=True) for col in columnas]
        datos.append(fila_datos)

# Asumiendo 38 jornadas y 10 partidos por jornada
num_jornadas = 38
partidos_por_jornada = len(datos) // num_jornadas

jornadas = []
for j in range(1, num_jornadas + 1):
    jornadas.extend([f"Jornada {j}"] * partidos_por_jornada)

# Si hay partidos extra (por ejemplo, partidos aplazados), añádelos a la última jornada
while len(jornadas) < len(datos):
    jornadas.append(f"Jornada {num_jornadas}")

columnas = ['Fecha', 'Hora', 'Local', 'Resultado', 'Visitante', 'Asistencia', 'Estadio']
df = pd.DataFrame(datos, columns=columnas)
df.insert(0, 'Jornada', jornadas)

df.to_csv('laliga_1995_1996.csv', index=False, encoding='utf-8')
print(df.head(15))

      Jornada       Fecha Hora        Local Resultado      Visitante  \
0   Jornada 1  02.09.1995         Deportivo         -       Valencia   
1   Jornada 1                      Sporting         -       Albacete   
2   Jornada 1  03.09.1995          Espanyol         -   UD Salamanca   
3   Jornada 1                     CP Mérida         -          Betis   
4   Jornada 1                      Athletic         -    Real Racing   
5   Jornada 1                      Atlético         -  Real Sociedad   
6   Jornada 1                       Sevilla         -       Tenerife   
7   Jornada 1                         Celta         -  SD Compostela   
8   Jornada 1                      Zaragoza         -    Real Oviedo   
9   Jornada 1                          Rayo         -    Real Madrid   
10  Jornada 1                    Valladolid         -      Barcelona   
11  Jornada 1  09.09.1995          Albacete         -        Sevilla   
12  Jornada 2                     Barcelona         -      CP Mé